In [1]:
import numpy as np
import scipy as sp
import scipy.linalg

In [2]:
def NKron(*args):
  """Calculate a Kronecker product over a variable number of inputs"""
  result = np.array([[1.0]])
  for op in args:
    result = np.kron(result, op)
  return result

In [3]:
zero = np.array([[1.0], [0.0]]) # |0>
one = np.array([[0.0], [1.0]]) # |1>
zerozerozero = NKron(zero, zero, zero) # |000>
oneoneone = NKron(one, one, one) # |000>

In [4]:
NormalizeState = lambda state: state / sp.linalg.norm(state)
plus = NormalizeState(zerozerozero + oneoneone) # (|000> + |111>)/sqrt(2)
minus = NormalizeState(zerozerozero - oneoneone) # (|000> - |111>)/sqrt(2)
zerol = NKron(plus, plus, plus)
onel = NKron(minus, minus, minus)

## Channel

In [5]:
X = np.array([[0.0, 1.0],[1.0, 0.0]])
Y = np.array([[0.0, 1j],[1j, 0.0]])
Z = np.array([[1.0, 0.0],[0.0, -1.0]])

In [6]:
Id = np.eye(2)
Flip0 = NKron(Id, Id, Id, Id, Id, Id, Id, Id, Id)

Flip1 = NKron(X, Id, Id, Id, Id, Id, Id, Id, Id)
Flip2 = NKron(Id, X, Id, Id, Id, Id, Id, Id, Id)
Flip3 = NKron(Id, Id, X, Id, Id, Id, Id, Id, Id)

Flip4 = NKron(Id, Id, Id, X, Id, Id, Id, Id, Id)
Flip5 = NKron(Id, Id, Id, Id, X, Id, Id, Id, Id)
Flip6 = NKron(Id, Id, Id, Id, Id, X, Id, Id, Id)

Flip7 = NKron(Id, Id, Id, Id, Id, Id, X, Id, Id)
Flip8 = NKron(Id, Id, Id, Id, Id, Id, Id, X, Id)
Flip9 = NKron(Id, Id, Id, Id, Id, Id, Id, Id, X)

In [7]:
PFlip1 = NKron(Z, Id, Id, Id, Id, Id, Id, Id, Id)
PFlip2 = NKron(Id, Z, Id, Id, Id, Id, Id, Id, Id)
PFlip3 = NKron(Id, Id, Z, Id, Id, Id, Id, Id, Id)
PFlip4 = NKron(Id, Id, Id, Z, Id, Id, Id, Id, Id)
PFlip5 = NKron(Id, Id, Id, Id, Z, Id, Id, Id, Id)
PFlip6 = NKron(Id, Id, Id, Id, Id, Z, Id, Id, Id)
PFlip7 = NKron(Id, Id, Id, Id, Id, Id, Z, Id, Id)
PFlip8 = NKron(Id, Id, Id, Id, Id, Id, Id, Z, Id)
PFlip9 = NKron(Id, Id, Id, Id, Id, Id, Id, Id, Z)

In [8]:
c1 = 0
c2 = 1
c3 = 0
c4 = 0
U = c1*np.eye(2) + c2*X + c3*np.matmul(X,Z) + c4*Z
U = U/np.sqrt(c1**2 + c2**2 + c3**2 + c4**2)

genFlip1 = NKron(U, Id, Id, Id, Id, Id, Id, Id, Id)
genFlip2 = NKron(Id, U, Id, Id, Id, Id, Id, Id, Id)
genFlip3 = NKron(Id, Id, U, Id, Id, Id, Id, Id, Id)
genFlip4 = NKron(Id, Id, Id, U, Id, Id, Id, Id, Id)
genFlip5 = NKron(Id, Id, Id, Id, U, Id, Id, Id, Id)
genFlip6 = NKron(Id, Id, Id, Id, Id, U, Id, Id, Id)

## |0>    -->    |0L>    -->   channel

In [9]:
rxQbit = (np.dot(genFlip6, zerol))

## Syndrome check

In [10]:
M1 = NKron(Z, Z, Id, Id, Id, Id, Id, Id, Id)
M2 = NKron(Z, Id, Z, Id, Id, Id, Id, Id, Id)

M3 = NKron(Id, Id, Id, Z, Z, Id, Id, Id, Id)
M4 = NKron(Id, Id, Id, Z, Id, Z, Id, Id, Id)

M5 = NKron(Id, Id, Id, Id, Id, Id, Z, Z, Id)
M6 = NKron(Id, Id, Id, Id, Id, Id, Z, Id, Z)

M7 = NKron(X, X, X, X, X, X, Id, Id, Id)
M8 = NKron(X, X, X, Id, Id, Id, X, X, X)

In [11]:
m1 = (np.dot(rxQbit.transpose(), np.dot(M1, rxQbit)))
m2 = (np.dot(rxQbit.transpose(), np.dot(M2, rxQbit)))

print(m1)
print(m2)

m1 = np.sign(np.dot(rxQbit.transpose(), np.dot(M1, rxQbit)))
m2 = np.sign(np.dot(rxQbit.transpose(), np.dot(M2, rxQbit)))

if m1 == 1 and m2 == 1:
    finalQbit = np.dot(Flip0, rxQbit)
elif m1 == 1 and m2 == -1:
    finalQbit = np.dot(Flip3, rxQbit)
elif m1 == -1 and m2 == 1:
    finalQbit = np.dot(Flip2, rxQbit)
elif m1 == -1 and m2 == -1:
    finalQbit = np.dot(Flip1, rxQbit)
        
m1 = (np.dot(rxQbit.transpose(), np.dot(M3, rxQbit)))
m2 = (np.dot(rxQbit.transpose(), np.dot(M4, rxQbit)))

print(m1)
print(m2)

m1 = np.sign(np.dot(rxQbit.transpose(), np.dot(M3, rxQbit)))
m2 = np.sign(np.dot(rxQbit.transpose(), np.dot(M4, rxQbit)))

if m1 == 1 and m2 == 1:
    finalQbit = np.dot(Flip0, finalQbit)
elif m1 == 1 and m2 == -1:
    finalQbit = np.dot(Flip6, finalQbit)
elif m1 == -1 and m2 == 1:
    finalQbit = np.dot(Flip5, finalQbit)
elif m1 == -1 and m2 == -1:
    finalQbit = np.dot(Flip4, finalQbit)

m1 = (np.dot(rxQbit.transpose(), np.dot(M5, rxQbit)))
m2 = (np.dot(rxQbit.transpose(), np.dot(M6, rxQbit)))

print(m1)
print(m2)

m1 = np.sign(np.dot(rxQbit.transpose(), np.dot(M5, rxQbit)))
m2 = np.sign(np.dot(rxQbit.transpose(), np.dot(M6, rxQbit)))

if m1 == 1 and m2 == 1:
    finalQbit = np.dot(Flip0, finalQbit)
elif m1 == 1 and m2 == -1:
    finalQbit = np.dot(Flip9, finalQbit)
elif m1 == -1 and m2 == 1:
    finalQbit = np.dot(Flip8, finalQbit)
elif m1 == -1 and m2 == -1:
    finalQbit = np.dot(Flip7, finalQbit)

m1 = (np.dot(rxQbit.transpose(), np.dot(M7, rxQbit)))
m2 = (np.dot(rxQbit.transpose(), np.dot(M8, rxQbit)))

print(m1)
print(m2)

m1 = np.sign(np.dot(rxQbit.transpose(), np.dot(M7, rxQbit)))
m2 = np.sign(np.dot(rxQbit.transpose(), np.dot(M8, rxQbit)))

if m1 == 1 and m2 == 1:
    finalQbit = np.dot(Flip0, finalQbit)
elif m1 == 1 and m2 == -1:
    finalQbit = np.dot(PFlip7, finalQbit)
elif m1 == -1 and m2 == 1:
    finalQbit = np.dot(PFlip4, finalQbit)
elif m1 == -1 and m2 == -1:
    finalQbit = np.dot(PFlip1, finalQbit)

[[1.]]
[[1.]]
[[1.]]
[[-1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]


In [12]:
np.sum(finalQbit - zerol)

0.0

In [13]:
finalQbit

array([[0.35355339],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.35355339],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.   

In [14]:
zerol

array([[0.35355339],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.35355339],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.   